<a href="https://colab.research.google.com/github/KingT5M/NARENDRA-CONCURRENT-FAULT/blob/main/NARENDRA_CONCURRENT_FAULT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt 
import keras_tuner as kt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from scipy.stats import randint, uniform 
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, MaxPooling1D, LSTM, Flatten, Dense, Dropout, Reshape
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, Callback
from kerastuner import HyperModel, HyperParameters
from keras.layers import Input, Dense, GRU
from keras.models import Model

# File paths
file_paths = [
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\DelayAPP.csv',
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\GainRPM.csv',
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\HealthyData.csv',
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\NoiseAPP.csv',
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\PacketLossAPP.csv'
]

# Load and preprocess the data
def load_and_preprocess_data(file_paths):
    data_frames = []
    for file_path in file_paths:
        if os.path.exists(file_path):
            data = pd.read_csv(file_path, header=None, usecols=[i for i in range(1, 10)])
            data_frames.append(data)
        else:
            print(f"File not found: {file_path}")
    if not data_frames:
        print("No data frames loaded.")
        return None
    concatenated_data = pd.concat(data_frames, ignore_index=True)
    print("Concatenated data shape:", concatenated_data.shape)
    concatenated_data = concatenated_data.sample(frac=1).reset_index(drop=True)
    print("Concatenated data shape after shuffling:", concatenated_data.shape)
    return concatenated_data

# Load and preprocess the data
concatenated_data = load_and_preprocess_data(file_paths)

# Preprocess the concatenated data
def preprocess_data(concatenated_data):
    features = concatenated_data.iloc[:, :-1].values
    labels = concatenated_data.iloc[:, -1].values
    
    # Label Encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    
    # One-Hot Encoding
    onehot_encoder = OneHotEncoder()
    labels_encoded = labels_encoded.reshape(-1, 1)
    labels_onehot = onehot_encoder.fit_transform(labels_encoded).toarray()
    
    return features, labels_onehot

# Create sequences
def create_sequences(features, labels, sequence_length=30):
    X, y = [], []
    for i in range(len(features) - sequence_length + 1):
        end_ix = i + sequence_length
        seq_x, seq_y = features[i:end_ix], labels[end_ix - 1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Preprocess the data
features, labels_onehot = preprocess_data(concatenated_data)

# Split data into train, validation, and test sets
sequence_length = 30
X_seq, y_seq = create_sequences(features, labels_onehot, sequence_length)

X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Denoising Autoencoder (DAE) for feature extraction
input_dim = X_train.shape[2]
latent_dim = 64

input_layer = Input(shape=(sequence_length, input_dim))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(latent_dim, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

dae = Model(input_layer, decoded)
dae.compile(optimizer='adam', loss='mean_squared_error')

dae.fit(X_train, X_train, epochs=50, batch_size=64, validation_split=0.2)

encoder = Model(input_layer, encoded)
features_dae_train = encoder.predict(X_train)
features_dae_val = encoder.predict(X_val)
features_dae_test = encoder.predict(X_test)

features_dae_train = np.array(features_dae_train)
features_dae_val = np.array(features_dae_val)
features_dae_test = np.array(features_dae_test)

# Gated Recurrent Unit (GRU) for feature learning
gru_input_shape = features_dae_train.shape[1:]

gru_model = Sequential()
gru_model.add(GRU(64, input_shape=gru_input_shape, return_sequences=True))
gru_model.add(GRU(64, return_sequences=True))
gru_model.add(GRU(64, return_sequences=True))
gru_model.add(GRU(64))

gru_model.add(Reshape((1, 64)))

gru_output_train = gru_model.predict(features_dae_train)
gru_output_val = gru_model.predict(features_dae_val)
gru_output_test = gru_model.predict(features_dae_test)

# Define metrics callback with REINFORCE algorithm
class MetricsCallback(Callback):
    def __init__(self, validation_data, fault_types):
        super(MetricsCallback, self).__init__()
        self.validation_data = validation_data
        self.fault_types = fault_types
        self.rewards = []

    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data
        y_pred = self.model.predict(X_val)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = np.argmax(y_val, axis=1)

        precision = precision_score(y_true, y_pred_classes, average='weighted')
        recall = recall_score(y_true, y_pred_classes, average='weighted')
        f1 = f1_score(y_true, y_pred_classes, average='weighted')

        # Calculate reward based on achieved metrics
        reward = (precision >= 0.9886 and recall >= 0.9890 and f1 >= 0.9888) - 1
        self.rewards.append(reward)

        print(f"Validation Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

        if precision >= 0.9886 and recall >= 0.9890 and f1 >= 0.9888:
            print("Achieved desired metrics. Stopping training.")
            self.model.stop_training = True
            
# Define fault_types
fault_types = ['delay-time', 'gain', 'healthy', 'noise', 'packetloss']

# Define hypermodel with REINFORCE
class CustomHyperModel(HyperParameters):
    def __init__(self, X_train, y_train, X_val, y_val, fault_types):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.fault_types = fault_types  

    def build(self, hp):
        model = Sequential()
        
        cnn_layers = hp.Int('cnn_layers', min_value=0, max_value=5, default=5)
        lstm_layers = hp.Int('lstm_layers', min_value=0, max_value=5, default=4)
        dense_layers = hp.Int('dense_layers', min_value=0, max_value=5, default=0)
        epochs = hp.Int('epochs', min_value=50, max_value=900, default=850)
        max_pooling = hp.Int('max_pooling', min_value=0, max_value=1, default=1)
        dropout = hp.Float('dropout', min_value=0, max_value=0.5, default=0)
        batch_norm = hp.Int('batch_norm', min_value=0, max_value=2, default=2)
        batch_size = hp.Int('batch_size', min_value=64, max_value=150, default=64)
        learning_rate = hp.Float('learning_rate', min_value=0.0001, max_value=0.001, default=0.0005, sampling='linear')

        for i in range(cnn_layers):
            model.add(Conv1D(filters=8, kernel_size=2, activation='relu', padding='same', input_shape=(sequence_length, latent_dim)))
            if batch_norm:
                model.add(BatchNormalization())

        if max_pooling:
            model.add(MaxPooling1D(pool_size=2))

        for i in range(lstm_layers):
            model.add(LSTM(units=64, activation='relu', return_sequences=True))
            if batch_norm:
                model.add(BatchNormalization())

        model.add(Flatten())

        for i in range(dense_layers):
            model.add(Dense(units=64, activation='relu'))
            if dropout:
                model.add(Dropout(dropout))
            if batch_norm:
                model.add(BatchNormalization())

        model.add(Dense(units=len(self.fault_types), activation='softmax'))

        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        return model

# Define parameters
num_episodes = 100
max_steps_per_episode = 50
current_hyperparameters = {
    'cnn_layers': 5,
    'lstm_layers': 4,
    'dense_layers': 0,
    'epochs': 850,
    'max_pooling': 1,
    'dropout': 0,
    'batch_norm': 2,
    'batch_size': 64,
    'learning_rate': 0.0005
}

print("Reinforcement Learning starts.")
# Training loop with REINFORCE
for episode in range(num_episodes):
    episode_rewards = []

    for step in range(max_steps_per_episode):
        sampled_hyperparameters = {
            'cnn_layers': int(np.random.normal(current_hyperparameters['cnn_layers'], 0.1)),
            'lstm_layers': int(np.random.normal(current_hyperparameters['lstm_layers'], 0.1)),
            'dense_layers': int(np.random.normal(current_hyperparameters['dense_layers'], 0.1)),
            'epochs': int(np.random.normal(current_hyperparameters['epochs'], 0.1)),
            'max_pooling': int(np.random.normal(current_hyperparameters['max_pooling'], 0.1)),
            'dropout': int(np.random.normal(current_hyperparameters['dropout'], 0.1)),
            'batch_norm': int(np.random.normal(current_hyperparameters['batch_norm'], 0.1)),
            'batch_size': int(np.random.normal(current_hyperparameters['batch_size'], 0.1)),
            'learning_rate': np.random.normal(current_hyperparameters['learning_rate'], 0.1)
        }
        hp = HyperParameters()
        # Define hyper-parameter ranges
        hp.Int('cnn_layers', min_value=0, max_value=5, default=5)
        hp.Int('lstm_layers', min_value=0, max_value=5, default=4)
        hp.Int('dense_layers', min_value=0, max_value=5, default=0)
        hp.Int('epochs', min_value=50, max_value=900, default=850)
        hp.Int('max_pooling', min_value=0, max_value=1, default=1)
        hp.Float('dropout', min_value=0, max_value=0.5, default=0)
        hp.Int('batch_norm', min_value=0, max_value=2, default=2)
        hp.Int('batch_size', min_value=64, max_value=150, default=64)
        hp.Float('learning_rate', min_value=0.0001, max_value=0.001, default=0.0005, sampling='linear')

        # Pass the HyperParameters object to the CustomHyperModel
        hypermodel = CustomHyperModel(features_dae_train, y_train, features_dae_val, y_val, fault_types)

        # Build the model with sampled hyperparameters
        model = hypermodel.build(hp)

        # Train the model
        history = model.fit(
            features_dae_train,
            y_train,
            validation_data=(features_dae_val, y_val),
            batch_size=sampled_hyperparameters['batch_size'],
            epochs=sampled_hyperparameters['epochs'],
            verbose=0
        )

        # Evaluate the model
        loss, accuracy = model.evaluate(features_dae_val, y_val, verbose=0)

        # Calculate reward based on accuracy
        reward = accuracy - 0.5  # Example reward function, you can adjust this based on your specific objectives

        # Append reward to episode rewards
        episode_rewards.append(reward)

    # Update hyperparameters using REINFORCE algorithm
    for hp_name, reward in zip(current_hyperparameters.keys(), episode_rewards):
        current_hyperparameters[hp_name] += 0.01 * reward  # Adjust the update factor as needed

    print(f"Episode {episode + 1}: Rewards - {np.mean(episode_rewards)}")

print("Reinforcement Learning stops.")

# Get the best hyperparameters
best_hyperparameters = current_hyperparameters
print("Best Hyperparameters:", best_hyperparameters)

# Build the final model with the best hyperparameters
best_hypermodel = CustomHyperModel(features_dae_train, y_train, features_dae_val, y_val, fault_types)
best_model = best_hypermodel.build(best_hyperparameters)

# Train the final model
history = best_model.fit(
    features_dae_train,
    y_train,
    validation_data=(features_dae_val, y_val),
    batch_size=best_hyperparameters['batch_size'],
    epochs=best_hyperparameters['epochs'],
    callbacks=[MetricsCallback((features_dae_val, y_val), fault_types)]
)

# Save the best model
best_model.save('best_model.h5')

# Calculate metrics on the test set
y_pred = best_model.predict(features_dae_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

precision = precision_score(y_true, y_pred_classes, average=None, zero_division=1)
recall = recall_score(y_true, y_pred_classes, average=None)
f1 = f1_score(y_true, y_pred_classes, average=None)

# Plotting precision, recall, and F1-score in one graph
x = np.arange(len(fault_types))
width = 0.2

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width, precision, width, label='Precision')
rects2 = ax.bar(x, recall, width, label='Recall')
rects3 = ax.bar(x + width, f1, width, label='F1-score')

ax.set_ylabel('Scores')
ax.set_title('Scores by fault types')
ax.set_xticks(x)
ax.set_xticklabels(fault_types)
ax.legend()

fig.tight_layout()

# Save the graph
plt.savefig('scores.png')


C:\Users\T5M\AppData\Local\Temp\ipykernel_9460\3921900960.py:17: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel, HyperParameters


Concatenated data shape: (42597, 9)
Concatenated data shape after shuffling: (42597, 9)


Epoch 1/50

426/426 [==============================] - 7s 12ms/step - loss: 875072640.0000 - val_loss: 875738112.0000
Epoch 2/50
426/426 [==============================] - 5s 11ms/step - loss: 875073152.0000 - val_loss: 875738112.0000
Epoch 3/50
426/426 [==============================] - 7s 15ms/step - loss: 875072960.0000 - val_loss: 875738112.0000
Epoch 4/50
426/426 [==============================] - 6s 15ms/step - loss: 875073024.0000 - val_loss: 875738112.0000
Epoch 5/50
108/426 [======>.......................] - ETA: 3s - loss: 875006784.0000